# 1. Setup and Load Data

In [1]:
import pandas as pd
import sqlite3

# Load data
df = pd.read_csv("../data/inventory_data.csv")
df.head()


,date,sku,demand,inventory_level,stockout,lead_time_days
0,2024-01-01,SKU_A,23,477,0,9
1,2024-01-02,SKU_A,19,458,0,5
2,2024-01-03,SKU_A,26,432,0,5
3,2024-01-04,SKU_A,25,407,0,6
4,2024-01-05,SKU_A,15,392,0,8


# 2. Create SQLite DB and Table

In [6]:
# Create a local SQLite database file inside SQLAnalytics/data
conn = sqlite3.connect("../data/sql_analytics.db")

# Save dataframe as a SQL table
df.to_sql("inventory", conn, if_exists="replace", index=False)

# Quick check
pd.read_sql("SELECT COUNT(*) AS row_count FROM inventory;", conn)

,row_count
0,360


In [7]:
pd.read_sql("""
SELECT sku, COUNT(*) AS days
FROM inventory
GROUP BY sku;
""", conn)


,sku,days
0,SKU_A,120
1,SKU_B,120
2,SKU_C,120


# Query 1 — Preview columns

In [8]:
pd.read_sql("SELECT * FROM inventory LIMIT 5;", conn)


,date,sku,demand,inventory_level,stockout,lead_time_days
0,2024-01-01,SKU_A,23,477,0,9
1,2024-01-02,SKU_A,19,458,0,5
2,2024-01-03,SKU_A,26,432,0,5
3,2024-01-04,SKU_A,25,407,0,6
4,2024-01-05,SKU_A,15,392,0,8


# Query 2 — Stockout rate (%) per SKU

In [9]:
pd.read_sql("""
SELECT
  sku,
  ROUND(AVG(stockout) * 100, 2) AS stockout_rate_pct
FROM inventory
GROUP BY sku
ORDER BY stockout_rate_pct DESC;
""", conn)


,sku,stockout_rate_pct
0,SKU_C,80.00
1,SKU_A,80.00
2,SKU_B,78.33


# Query 3 — Average demand per SKU

In [10]:
pd.read_sql("""
SELECT
  sku,
  ROUND(AVG(demand), 2) AS avg_demand
FROM inventory
GROUP BY sku
ORDER BY avg_demand DESC;
""", conn)


,sku,avg_demand
0,SKU_A,20.16
1,SKU_C,20.01
2,SKU_B,19.57


# Query 4 — Demand variability proxy (variance) per SKU

In [11]:
pd.read_sql("""
SELECT
  sku,
  ROUND(AVG(demand * demand) - AVG(demand) * AVG(demand), 2) AS demand_variance
FROM inventory
GROUP BY sku
ORDER BY demand_variance DESC;
""", conn)


,sku,demand_variance
0,SKU_B,19.33
1,SKU_C,17.97
2,SKU_A,17.25


# Query 5 — Days with inventory at zero (service risk)

In [12]:
pd.read_sql("""
SELECT
  sku,
  SUM(CASE WHEN inventory_level = 0 THEN 1 ELSE 0 END) AS zero_inventory_days
FROM inventory
GROUP BY sku
ORDER BY zero_inventory_days DESC;
""", conn)


,sku,zero_inventory_days
0,SKU_C,96
1,SKU_A,96
2,SKU_B,94


# Query 6 — Top 10 highest-demand days (overall)

In [13]:
pd.read_sql("""
SELECT date, sku, demand
FROM inventory
ORDER BY demand DESC
LIMIT 10;
""", conn)


,date,sku,demand
0,2024-02-12,SKU_A,32
1,2024-01-06,SKU_C,31
2,2024-02-02,SKU_B,30
3,2024-02-24,SKU_B,30
4,2024-03-13,SKU_C,30
5,2024-02-23,SKU_A,29
6,2024-02-24,SKU_A,29
7,2024-04-16,SKU_A,29
8,2024-02-12,SKU_C,29
9,2024-01-22,SKU_A,28


# Query 7 — Monthly demand trend per SKU

In [14]:
pd.read_sql("""
SELECT
  substr(date, 1, 7) AS month,
  sku,
  SUM(demand) AS total_demand
FROM inventory
GROUP BY month, sku
ORDER BY month, sku;
""", conn)


,month,sku,total_demand
0,2024-01,SKU_A,614
1,2024-01,SKU_B,578
2,2024-01,SKU_C,616
3,2024-02,SKU_A,607
4,2024-02,SKU_B,602
5,2024-02,SKU_C,556
6,2024-03,SKU_A,600
7,2024-03,SKU_B,604
8,2024-03,SKU_C,646
9,2024-04,SKU_A,598


# Query 8 — Lead time average per SKU

In [15]:
pd.read_sql("""
SELECT
  sku,
  ROUND(AVG(lead_time_days), 2) AS avg_lead_time_days
FROM inventory
GROUP BY sku
ORDER BY avg_lead_time_days DESC;
""", conn)


,sku,avg_lead_time_days
0,SKU_B,6.10
1,SKU_A,5.96
2,SKU_C,5.92


# Query 9 — Inventory health snapshot (avg + min inventory)

In [16]:
pd.read_sql("""
SELECT
  sku,
  ROUND(AVG(inventory_level), 2) AS avg_inventory,
  MIN(inventory_level) AS min_inventory
FROM inventory
GROUP BY sku;
""", conn)


,sku,avg_inventory,min_inventory
0,SKU_A,50.27,0
1,SKU_B,52.19,0
2,SKU_C,50.18,0


# Query 10 — “Worst week” stockouts per SKU (counts)

In [17]:
pd.read_sql("""
SELECT
  sku,
  substr(date, 1, 10) AS day,
  stockout
FROM inventory
LIMIT 5;
""", conn)


,sku,day,stockout
0,SKU_A,2024-01-01,0
1,SKU_A,2024-01-02,0
2,SKU_A,2024-01-03,0
3,SKU_A,2024-01-04,0
4,SKU_A,2024-01-05,0


## Conclusion

This SQL analysis demonstrates how structured queries can be used to
derive key inventory and demand insights from operational data.

By calculating stockout rates, demand variability, and inventory health indicators,
the analysis highlights SKUs with higher operational risk and supports
data-driven inventory planning decisions.

The results show how SQL can be effectively applied in analytics engineering
and business reporting contexts to support supply chain decision-making.